#**FEATURE ENGINEERING AND DATA PREPROCESSING**

###creating the final dataframe by merging all the dataframe and also performing downcasting to reduce memory consumption

In [ ]:
#REFERENCE-https://www.kaggle.com/priyanka4pc/m5-model
#code to reduce memory consumption
import numpy as np
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
calendar=pd.read_csv("/content/calendar.csv")
import datetime as dt
calendar['date'] = calendar['date'].apply(lambda x: 
                                    dt.datetime.strptime(x,'%Y-%m-%d'))

#REFERNCE-https://stackoverflow.com/questions/65842209/how-to-downcast-numeric-columns-in-pandas
#int_columns = calendar.select_dtypes('integer').columns#selecting columns with data type int
#calendar[int_columns] = calendar[int_columns].apply(pd.to_numeric,downcast='integer')#downcasting int64 datatype columns

object_columns=calendar.select_dtypes('object').columns
calendar[object_columns]=calendar[object_columns].astype('category')

calendar=reduce_mem_usage(calendar)

Mem. usage decreased to  0.14 Mb (38.6% reduction)


In [ ]:
price=pd.read_csv("/content/sell_prices.csv")

#REFERNCE-https://stackoverflow.com/questions/65842209/how-to-downcast-numeric-columns-in-pandas
#int_columns=price.select_dtypes('integer').columns
#price[int_columns]=price[int_columns].apply(pd.to_numeric,downcast='integer')

#float_columns=price.select_dtypes('float').columns
#price[float_columns]=price[float_columns].apply(pd.to_numeric,downcast='float')

#REFERENCE-https://stackoverflow.com/questions/39092067/pandas-dataframe-convert-column-type-to-string-or-categorical
object_columns=price.select_dtypes('object').columns
price[object_columns]=price[object_columns].astype('category')

price=reduce_mem_usage(price)

Mem. usage decreased to 45.77 Mb (63.1% reduction)


In [ ]:
sales=pd.read_csv("/content/sales_train_evaluation.csv")


#REFERENCE-https://stackoverflow.com/questions/39092067/pandas-dataframe-convert-column-type-to-string-or-categorical
object_columns=sales.select_dtypes('object').columns
sales[object_columns]=sales[object_columns].astype('category')

#float_columns=sales.select_dtypes('float').columns
#sales[float_columns]=sales[float_columns].apply(pd.to_numeric,downcast='float')

sales=reduce_mem_usage(sales)



#dataframe is melted to put all the sales data under a single column,it will help us in plotting the time series data of sale..
sales_melt=sales.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d',value_name='sale')

sales_melt['d']=sales_melt['d'].astype('category')

Mem. usage decreased to 96.55 Mb (78.7% reduction)


In [ ]:
#merging sales data with calendar data to plot total sales per day(in terms of date)
sales_pivot=sales_melt.merge(calendar,on='d',how='left')
sales_pivot.head()

#converting column d to category datatype tp reduce memory consumption
sales_pivot['d']=sales_pivot['d'].astype('category')


In [ ]:
final_df=sales_pivot.merge(price,on=['store_id', 'item_id', 'wm_yr_wk'],how='left')
#REFERNCE-https://stackoverflow.com/questions/65842209/how-to-downcast-numeric-columns-in-pandas
#int_columns = final_df.select_dtypes('integer').columns#selecting columns with data type int
#final_df[int_columns] = final_df[int_columns].apply(pd.to_numeric,downcast='integer')#downcasting int64 datatype columns

#REFERENCE-https://stackoverflow.com/questions/39092067/pandas-dataframe-convert-column-type-to-string-or-categorical
object_columns=final_df.select_dtypes('object').columns
final_df[object_columns]=final_df[object_columns].astype('category')

#float_columns=final_df.select_dtypes('float').columns
#final_df[float_columns]=final_df[float_columns].apply(pd.to_numeric,downcast='float')

final_df=reduce_mem_usage(final_df)
final_df.head()



Mem. usage decreased to 2485.02 Mb (0.0% reduction)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sale,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN


In [ ]:
final_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59181090 entries, 0 to 59181089
Data columns (total 22 columns):
 #   Column        Non-Null Count     Dtype         
---  ------        --------------     -----         
 0   id            59181090 non-null  category      
 1   item_id       59181090 non-null  category      
 2   dept_id       59181090 non-null  category      
 3   cat_id        59181090 non-null  category      
 4   store_id      59181090 non-null  category      
 5   state_id      59181090 non-null  category      
 6   d             59181090 non-null  category      
 7   sale          59181090 non-null  int16         
 8   date          59181090 non-null  datetime64[ns]
 9   wm_yr_wk      59181090 non-null  int16         
 10  weekday       59181090 non-null  category      
 11  wday          59181090 non-null  int8          
 12  month         59181090 non-null  int8          
 13  year          59181090 non-null  int16         
 14  event_name_1  4817420 non-null  

##**OBSERVATION-**
    THE COLUMNS EVENT_NAME_1,EVENT_TYPE_1,EVENT_NAME_2,EVENT_TYPE_2 AND SELL_PRICE CONTAINS NAN VALUES..

##SOME PREPROCESSING OF COLUMN 'd'

In [ ]:
#HERE WE ARE STRIPPING THE D FROM D COLUMN AND MAKING IT INTEGER...IT WILL HELP US TO SPLIT THE DATASET USING DAY
final_df['day'] = final_df['d'].map(lambda x: x.split('_')[1]).astype('int')


In [ ]:
#BEFORE SPLITTING THE DATASET WE ARE REMOVING THE WEEKDAY COLUMN AS WE ALREADY HAVE WDAY COLUMN WHICH IS NUMERIC REPRESENTATION OF WEEKDAY..
#WE ARE ALSO DROPING 'D' COLUMN SINCE WE HAVE CONVERTED IT TO A COLUMN 'DAY'
final_df=final_df.drop(['weekday','d'],axis=1)


In [ ]:
final_df=reduce_mem_usage(final_df)

Mem. usage decreased to 2428.48 Mb (12.2% reduction)


##**DEALING WITH THE NAN VALUES-**

###DEALING WITH NAN VALUES IN SELL_PRICE COLUMN

In [ ]:
#out of 59181090 rows how many contains null value
final_df['sell_price'].isna().sum()

12299413

### 1ST APPROACH FOR CREATING BASELINE MODEL-

    REPLACE ALL NAN VALUES WITH ZERO

### 2ND APPROACH WHICH CAN BE TRIED TO IMPROVE OUR METRIC
    THE NAN VALUES ARE DUE TO THE FACT THAT THE ITEMS ARE NOT SOLD IN THAT WEEK..SO WE CAN REPLACE THE VALUES WITH THE MEADIAN SELL_PRICE OF THAT SAME ITEMS SOLD IN DIFFERENT WEEK 

In [ ]:
#REFERENCE-https://stackoverflow.com/questions/19966018/pandas-filling-missing-values-by-mean-in-each-group
#X_train['sell_price'] = X_train['sell_price'].fillna(X_train.groupby('id')['sell_price'].transform('median'))#FILLING NAN VALUES BY THE MEDIAN

#X_cv['sell_price'] = X_cv['sell_price'].fillna(X_cv.groupby('id')['sell_price'].transform('median'))#FILLING NAN VALUES BY THE MEDIAN

#X_test['sell_price'] = X_test['sell_price'].fillna(X_test.groupby('id')['sell_price'].transform('median'))#FILLING NAN VALUES BY THE MEDIAN

In [ ]:
final_df['sell_price'] = final_df['sell_price'].fillna(final_df.groupby('id')['sell_price'].transform('median'))



###DEALING WITH NAN VALUES IN EVENT_NAME_1,EVENT_TYPE_1,EVENT_NAME_2,EVENT_TYPE_2 COLUMNS/ENCODING CATEGORICAL FEATURE

###THE NAN VALUE IN THESE COLUMNS CAN EASILY BE DEALT BY ENCODING THE FEATURES USING LABEL ENCODINGS

In [ ]:
#REFERENCE-https://stackoverflow.com/questions/32011359/convert-categorical-data-in-pandas-dataframe
#I tried using label encoder but it was giving memory error...
#on serching found the stackoverflow article and since out categorical data are already in category dtype so we can use dataframe.cat.code to get th einteger code for each categorical value
#for features which are not of type category will throw error,so using try except to ignore that error..
for i in final_df.columns:
    try:
        final_df[i] = final_df[i].cat.codes
    except AttributeError:
        pass

##**FEATURE ENGINEERING--**

##EXTRACTING SOME FEATURES USING DATETIME

In [ ]:
final_df.drop(['wm_yr_wk'],axis=1,inplace=True)
final_df['week_of_year']=final_df['date'].dt.week

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


##**1.-**1ST ENGINEERED FEATURE CAN BE A BOOLEAN FEATURE INDICATING WHETHER A DAY FALLS ON WEEKEND(1) OR WEEKDAYS(0)...THE REASON BEHIND THIS IS DURING EDA STAGE IT WAS FOUND THAT SALES ARE HIGHER ON WEEKEND THAN ON WEEKDAYS..THUS WEEKEND DOES AFFECT THE SALES

In [ ]:
final_df['is_weekend']=final_df['wday'].map(lambda x:1 if x<=2 else 0)


##**2.**THE 2ND ENGINEERED FEATURE IS INDICATING 1 IF THE DAY IS THE 1ST DAY OF MONTH AND 2 IF THE LAST DAY OF MONTH AND 0 IF ITS NONE OF THEM....THE REASON IS BECAUSE AT EDA STAGE IT WAS FOUND OUT THAT SALES ARE HIGHER ON 1ST DAY OF MONTH AND MUCH LESS ON LAST DAY OF MONTH

In [ ]:
month_start=final_df['date'].dt.is_month_start.map(lambda x:1 if x==True else 0)
month_end=final_df['date'].dt.is_month_end.map(lambda x:2 if x==True else 0)

In [ ]:
final_df['is_month_start_or_end']=month_start+month_end


In [ ]:
del month_start
del month_end

##**3.**THE 3RD ENGINEERED FEATURE IS INDICATING 1 IF THE DAY IS THE 1ST 15 DAY OF MONTH AND 0 IF THE LAST 15 DAY OF MONTH...THE REASON IS BECAUSE AT EDA STAGE IT WAS FOUND OUT THAT SALES ARE HIGHER ON 1ST 15 DAY OF MONTH AND THE SALES DECREASES FOR LAST 15 DAYS.

In [ ]:
final_df['is_first15_day_or_last15_day']=final_df['date'].dt.day.map(lambda x:1 if x<=15 else 0)




##**4.**THE 4TH ENGINEERED FEATURE THAT CAN BE TRIED IS WHETHER THE DAY IS CHRISTMAS OR NOT..THE REASON IS BECAUSE IT WAS FOUND OUT THAT AT CHRISTMAS THE SALES ARE ZERO ALL OVER THE STORES..SO IT CAN BE A USEFUL FEATURE

In [ ]:
final_df.drop(['date'],axis=1,inplace=True)

In [ ]:
final_df['is_christmas']=final_df['event_name_1'].map(lambda x:1 if x==1 else 0)


In [ ]:
final_df=reduce_mem_usage(final_df)

Mem. usage decreased to 2144.70 Mb (47.9% reduction)


##**5.**THE 5TH ENGINEERED FEATURE WILL BE THE LAG FEATURE WHICH CONTRIBUTES THE MOST TO TIME SERIES PROBLEM..

##WE ARE TRYING WITH LAG OF 1,7,14,21,28,30 DAYS

In [ ]:
#REFERENCE-https://www.analyticsvidhya.com/blog/2019/12/6-powerful-feature-engineering-techniques-time-series/
from tqdm import tqdm
lags=[1,7,14,21,28,30,31]
for lag in tqdm(lags):
    final_df["lag_" + str(lag)] = final_df.groupby("id")["sale"].shift(lag)
    final_df["lag_" + str(lag)]=final_df["lag_" + str(lag)].fillna(0).astype('int')

100%|██████████| 7/7 [00:26<00:00,  3.80s/it]


In [ ]:
#filling nan values with 0 which was created due to lag features
final_df['lag_1']=final_df['lag_1'].fillna(0).astype('int')
final_df['lag_7']=final_df['lag_7'].fillna(0).astype('int')
final_df['lag_14']=final_df['lag_14'].fillna(0).astype('int')
final_df['lag_21']=final_df['lag_21'].fillna(0).astype('int')
final_df['lag_28']=final_df['lag_28'].fillna(0).astype('int')
final_df['lag_30']=final_df['lag_30'].fillna(0).astype('int')
final_df['lag_31']=final_df['lag_31'].fillna(0).astype('int')

In [ ]:
final_df=reduce_mem_usage(final_df)

Mem. usage decreased to 2934.85 Mb (44.7% reduction)


##**6.**THE NEXT FEATURE IS THE ROLLING MEAN OF SALES WITH DIFFERENT WINDOW SIZE

In [ ]:
import numpy as np
import multiprocessing
from datetime import datetime
start=datetime.now()
temp=final_df[['id','sale']]
def rolling_7(temp):
      print("7 start")
      rolling_mean_7 = temp.groupby(['id'])['sale'].transform(lambda x: x.rolling(7).mean())
      rolling_mean_7.to_csv("rolling_mean_7")
      print("7 complete")

      

def rolling_14(temp):
      print("14 start")
      rolling_mean_14 = temp.groupby(['id'])['sale'].transform(lambda x: x.rolling(14).mean())
      rolling_mean_14.to_csv("rolling_mean_14")
      print("14 complete")
      
     


      


def rolling_28(temp):
      print("28 start")
      rolling_mean_28 = temp.groupby(['id'])['sale'].transform(lambda x: x.rolling(28).mean())
      rolling_mean_28.to_csv("rolling_mean_28")
      print("28 complete")
      
      

p1 = multiprocessing.Process(target=rolling_7,args=(temp,))
p2 = multiprocessing.Process(target=rolling_14,args=(temp,))
p3 = multiprocessing.Process(target=rolling_28,args=(temp,))




p1.start()
p2.start()
p3.start()


p1.join()
p2.join()
p3.join()


end=datetime.now()
difference=end-start
print("time taken",difference)


7 start
14 start
28 start
7 complete
14 complete
28 complete
time taken 1:02:39.128813


In [ ]:
import os
os.listdir()

In [ ]:
rolling=pd.read_csv("rolling_mean_28")


In [ ]:
del rolling

In [ ]:
rolling_mean_7=pd.read_csv("rolling_mean_7",)
final_df['rolling_mean_7']=rolling_mean_7.drop(['Unnamed: 0'],axis=1)


rolling_mean_14=pd.read_csv("rolling_mean_14")
final_df['rolling_mean_14']=rolling_mean_14.drop(['Unnamed: 0'],axis=1)



rolling_mean_28=pd.read_csv("rolling_mean_28")
final_df['rolling_mean_28']=rolling_mean_28.drop(['Unnamed: 0'],axis=1)



In [ ]:
del rolling_mean_14
del rolling_mean_28
del rolling_mean_7

In [ ]:
final_df['rolling_mean_7']=final_df['rolling_mean_7'].fillna(0)
final_df['rolling_mean_14']=final_df['rolling_mean_14'].fillna(0)
#final_df['rolling_mean_21']=final_df['rolling_mean_21'].fillna(0)
final_df['rolling_mean_28']=final_df['rolling_mean_28'].fillna(0)

In [ ]:
final_df=reduce_mem_usage(final_df)

Mem. usage decreased to 3273.49 Mb (23.7% reduction)


In [ ]:
final_df.to_hdf('final_df.hdf','mydata',mode='w')

#df = pd.read_hdf('my_filename.hdf','mydata')

In [ ]:
final_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,sale,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,day,week_of_year,is_weekend,is_month_start_or_end,is_first15_day_or_last15_day,is_christmas,lag_1,lag_7,lag_14,lag_21,lag_28,lag_30,lag_31,rolling_mean_7,rolling_mean_14,rolling_mean_28
0,14370,1437,3,1,0,0,0,1,1,2011,-1,-1,-1,-1,0,0,0,8.257812,1,4,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0
1,14380,1438,3,1,0,0,0,1,1,2011,-1,-1,-1,-1,0,0,0,3.970703,1,4,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0
2,14390,1439,3,1,0,0,0,1,1,2011,-1,-1,-1,-1,0,0,0,2.970703,1,4,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0
3,14400,1440,3,1,0,0,0,1,1,2011,-1,-1,-1,-1,0,0,0,4.640625,1,4,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0
4,14410,1441,3,1,0,0,0,1,1,2011,-1,-1,-1,-1,0,0,0,2.980469,1,4,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0


##**7.**THE NEXT FEATURE IS THE ROLLING STANDARD DEVIATION OF SALES WITH DIFFERENT WINDOW SIZE


In [ ]:
start=datetime.now()
temp=final_df[['id','sale']]
def rolling1_7(temp):
      
      rolling_std_7 = temp.groupby(['id'])['sale'].transform(lambda x: x.rolling(7).std())
      rolling_std_7.to_csv("rolling_std_7")

      

def rolling1_14(temp):
      rolling_std_14 = temp.groupby(['id'])['sale'].transform(lambda x: x.rolling(14).std())
      rolling_std_14.to_csv("rolling_std_14")
      
     


#def rolling1_21(temp):
      #rolling_std_21 = temp.groupby(['id'])['sale'].transform(lambda x: x.rolling(21).std())
      #rolling_std_21.to_csv("rolling_std_21")
      
      


def rolling1_28(temp):
      rolling_std_28 = temp.groupby(['id'])['sale'].transform(lambda x: x.rolling(28).std())
      rolling_std_28.to_csv("rolling_std_28")
      
      

p1 = multiprocessing.Process(target=rolling1_7,args=(temp,))
p2 = multiprocessing.Process(target=rolling1_14,args=(temp,))
p3 = multiprocessing.Process(target=rolling1_28,args=(temp,))
#p3 = multiprocessing.Process(target=rolling1_21,args=(temp,))



p1.start()
p2.start()
p3.start()
#p4.start()

p1.join()
p2.join()
p3.join()
#p4.join()

end=datetime.now()
difference=end-start
print("time taken",difference)

time taken 1:03:54.940175


In [ ]:
rolling_std_7=pd.read_csv("rolling_std_7")
final_df['rolling_std_7']=rolling_std_7.drop(['Unnamed: 0'],axis=1)

rolling_std_14=pd.read_csv("rolling_std_14")
final_df['rolling_std_14']=rolling_std_14.drop(['Unnamed: 0'],axis=1)

#rolling_std_21=pd.read_csv("rolling_std_21")
#final_df['rolling_std_21']=rolling_std_21.drop(['Unnamed: 0'],axis=1)

rolling_std_28=pd.read_csv("rolling_std_28")
final_df['rolling_std_28']=rolling_std_28.drop(['Unnamed: 0'],axis=1)


In [ ]:
del rolling_std_7
del rolling_std_14
del rolling_std_28

In [ ]:
final_df=final_df.fillna(0)

In [ ]:
final_df=reduce_mem_usage(final_df)

Mem. usage decreased to 3612.13 Mb (0.0% reduction)


In [ ]:
final_df.head(10)

##**8.**SOME FEATURE ENGINEERING FROM THE PRICE DATA

##PREVIOUS DAY PRICE OF ITEMS(PRICE LAG WITH WINDOW 1)

In [ ]:
final_df["price_previous_day"]=final_df['sell_price'].transform(lambda x:x.shift(1))

In [ ]:
final_df['price_previous_day']=final_df['price_previous_day'].fillna(0)

##PRICE CHANGE FROM PREVIOUS DAY

In [ ]:
diff=(final_df['price_previous_day']-final_df['sell_price'])
deno=final_df['price_previous_day'].apply(lambda x:0.1 if x==0 else x)


In [ ]:
final_df['price_change']=diff/deno

In [ ]:
final_df['price_change']=final_df['price_change'].fillna(0)

In [ ]:
final_df=reduce_mem_usage(final_df)

Mem. usage decreased to 3837.89 Mb (8.1% reduction)


##COPYING THE FINAL DATAFRAME TO GOGGLE DRIVE TO REUSE LATER

In [ ]:
final_df.to_hdf('final_df.hdf','mydata',mode='w')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
!cp final_df.hdf '/content/gdrive/My Drive/m5_forecasting_data/'
!ls -lt '/content/gdrive/My Drive/m5_forecasting_data/'

total 3930004
-rw------- 1 root root 4024323408 Jul  1 10:02 final_df.hdf
